In [1]:
import cmasher as cmr
import janitor
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from typing import Callable, List, Tuple, Optional
import matplotlib.ticker as ticker
from matplotlib.patches import Patch

import numpy as np
import pandas as pd
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

sns.set_theme(
    context="notebook", font_scale=1.5, style="whitegrid", palette="gray"
)  #  Darkgrid Whitegrid Dark White Ticks

from utilities import save_mpl_fig

df_stars = pd.read_csv("../input/repo_stars_timeseries.csv").assign(
    date=lambda df: pd.to_datetime(df["date"])
)
df_stars

fileslug       date  stars  treated  boughtstars  \
0      0riion_py-sls-lambda-toolkit 2023-05-09      0        1            1   
1      0riion_py-sls-lambda-toolkit 2023-05-10      0        1            1   
2      0riion_py-sls-lambda-toolkit 2023-05-11      0        1            1   
3      0riion_py-sls-lambda-toolkit 2023-05-12      0        1            1   
4      0riion_py-sls-lambda-toolkit 2023-05-13     25        1            1   
...                             ...        ...    ...      ...          ...   
17545                zjupgx_diverge 2023-06-03      0        0            0   
17546                zjupgx_diverge 2023-06-04      0        0            0   
17547                zjupgx_diverge 2023-06-05      0        0            0   
17548                zjupgx_diverge 2023-06-06      0        0            0   
17549                zjupgx_diverge 2023-06-07      0        0            0   

       treated2     _merge  
0             2       both  
1             2       both  
2             2       both  
3             2       both  
4             2       both  
...         ...        ...  
17545         0  left_only  
17546         0  left_only  
17547         0  left_only  
17548         0  left_only  
17549         0  left_only  

[17550 rows x 7 columns]

In [7]:
# function
def plot_treated_vs_control(
    df: pd.DataFrame,
    date_col: str,
    value_col: str,
    treatment_col: str,
    estimator: str = "mean",
    ci: Optional[str] = None,
    annotate_dates: Optional[List[Tuple[str, str]]] = None,
    figsize_scale: float = 0.8,
    aspect_ratio: Tuple[int, int] = (12, 8),
    hue_order: List[int] = [2, 1, 0],
    style_order: List[int] = [2, 1, 0],
    palette_range: Tuple[float, float] = (0.5, 1),
    y_label: str = "Number of GitHub Stars",
    y_tick_formatter: Optional[Callable] = None,
    y_bins: int = 6,
    shade_start_date: Optional[str] = None,
    shade_end_date: Optional[str] = None,
    shade_color: str = ".6",
    shade_alpha: float = 0.5,
    shade_label: str = "Treatment period\n(12 May\u201319 May)",
    legend_order: List[int] = [2, 1, 0, 3],
    custom_labels: Optional[List[str]] = [
        "Control",
        "Treated (low dosage)",
        "Treated (high dosage)",
        "Treatment period\n(May 12\u201321)",
    ],
) -> None:
    """
    Plot treated vs control with a specified estimator function.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the data to plot.
    date_col : str
        Name of the column containing the date values.
    value_col : str
        Name of the column containing the values to plot.
    treatment_col : str
        Name of the column containing the treatment indicators.
    estimator : Callable, optional
        Estimator function to use for plotting, by default sns.lineplot.
    ci : str, optional
        Confidence interval to display, by default None.
    annotate_dates : List[Tuple[str, str]], optional
        List of tuples with start and end dates for annotations, by default None.
    figsize_scale : float, optional
        Scale for the figure size, by default 0.8.
    aspect_ratio : Tuple[int, int], optional
        Aspect ratio for the figure, by default (12, 8).
    hue_order : List[int], optional
        Order of the hues, by default [2, 1, 0].
    style_order : List[int], optional
        Order of the styles, by default [2, 1, 0].
    palette_range : Tuple[float, float], optional
        Range for the palette, by default (0.5, 1).
    y_label : str, optional
        Label for the y-axis, by default "Number of PyPI downloads".
    y_tick_formatter : Callable, optional
        Formatter for the y-axis ticks, by default None.
    y_bins : int, optional
        Number of bins for the y-axis, by default 6.
    shade_start_date : str, optional
        Start date for shading, by default None.
    shade_end_date : str, optional
        End date for shading, by default None.
    shade_color : str, optional
        Color for shading, by default ".6".
    shade_alpha : float, optional
        Alpha value for shading, by default 0.5.
    shade_label : str, optional
        Label for shading, by default "Treatment period\n(12 May\u201319 May)".
    legend_order : List[int], optional
        Order for the legend items, by default [2, 1, 0, 3].
    custom_labels : List[str], optional
        Custom labels for the legend, by default None.

    Returns
    -------
    None
    """
    figsize = tuple(k * figsize_scale for k in aspect_ratio)
    _, ax = plt.subplots(figsize=figsize)

    sns.lineplot(
        data=df,
        x=date_col,
        y=value_col,
        markersize=12,
        hue=treatment_col,
        style=treatment_col,
        hue_order=hue_order,
        style_order=style_order,
        ci=ci,
        estimator=estimator,
        palette=cmr.get_sub_cmap("Greys", *palette_range),
        #         palette=["navy", "navy", "0"],
        ax=ax,
    )

    myFmt = mdates.DateFormatter("%-d %b")
    ax.xaxis.set_major_formatter(myFmt)
    ax.xaxis.set_major_locator(plt.MaxNLocator(10))

    if shade_start_date and shade_end_date:
        ax.axvspan(
            pd.to_datetime(shade_start_date),
            pd.to_datetime(shade_end_date),
            color=shade_color,
            alpha=shade_alpha,
            label=shade_label,
        )

    plt.locator_params(axis="y", nbins=y_bins)
    if y_tick_formatter:
        ax.get_yaxis().set_major_formatter(ticker.FuncFormatter(y_tick_formatter))
    else:
        ax.get_yaxis().set_major_formatter(
            ticker.FuncFormatter(lambda x, p: format(int(x), ","))
        )

    ax.set_ylabel(y_label, fontsize=15)
    ax.set_xlabel("")

    handles, labels = ax.get_legend_handles_labels()
    shade_patch = Patch(color="grey", alpha=0.5, label="Shaded Area")
    handles.append(shade_patch)

    if not custom_labels:
        custom_labels = [
            "Control",
            "Treated (low dosage)",
            "Treated (high dosage)",
            shade_label,
        ]

    ax.legend(
        handles=[handles[idx] for idx in legend_order],
        labels=[custom_labels[idx] for idx in legend_order],
        title="",
        loc="upper left",
        fontsize=16,
        markerscale=1.5,
        markerfirst=True,
        frameon=False,
        handlelength=4,
    )

    plt.tight_layout()

In [8]:
common_opts = dict(
    df=df_stars,
    date_col="date",
    value_col="stars",
    treatment_col="treated2",
    shade_start_date="2023-05-12",
    shade_end_date="2023-05-21",
)

In [9]:
plot_treated_vs_control(**common_opts)
save_mpl_fig("../output/timeseries-stars-treated-012")

In [10]:
plot_treated_vs_control(**common_opts, estimator=np.median)
save_mpl_fig("../output/timeseries-stars-treated-012-medians")

In [11]:
df_stars.query("treated2==0")["fileslug"].nunique()

485

In [12]:
df_stars.query("treated2==1")["fileslug"].nunique()

75

In [13]:
df_stars.query("treated2==2")["fileslug"].nunique()

25